## 4장 다변량 선형회귀

In [2]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

2.0.0-alpha0


* 이전까지는 선형회귀(linear regression)이라고 하면 y = ax + b 의 개념으로 봤다.
>
$$ H(x) = w x + b $$

* 이제는 '다변량' 이라는 개념으로 식을 봐야 한다. y = ax1 + bx2 + cx3 + d 이런 느낌
$$ H(x_1, x_2, x_3) = w_1 x_1 + w_2 x_2 + w_3 x_3 + b $$

* 변수가 많아지면서 기존의 방식으로는 계산을 진행하기 어렵다.
* 그래서 Matrix의 개념을 도입하는 거다.

$$ w_1 x_1 + w_2 x_2 + w_3 x_3 $$
>
$$ = \begin{pmatrix} w_{ 1 } & w_{ 2 } & w_{ 3 } \end{pmatrix}\cdot \begin{pmatrix} x_{ 1 } \\ x_{ 2 } \\ x_{ 3 } \end{pmatrix} $$
>
$$ = WX $$

#### 그런데 W * X와 X * W 중 뭐가 더 나을까?

* instance는 여러 값이 나오지만, weight는 고정적이다.
* 그래서 X * W로 보는 게 깔끔(?!)
* Tensorflow는 H(X) = XW로 계산

### 예시 도전!

* 함수식과 Matrix를 직접 비교해보자

x1 | x2 | y
---- | ---- | ----
1  |  0  |  1
0  |  2  |  2
3  |  0  |  3
0  |  4  |  4
5  |  0  |  5

### 함수식

In [5]:
tf.random.set_seed(0)  # 계산 값 동일하도록 seed값 설정

In [6]:
x1 = [1, 0, 3, 0, 5]
x2 = [0, 2, 0, 4, 0]
y  = [1, 2, 3, 4, 5]

In [8]:
W1 = tf.Variable(tf.random.uniform((1,), -10.0, 10.0)) # uniform(균등)한 형태로 -10부터 10사이에서 랜덤한 변수 설정
W2 = tf.Variable(tf.random.uniform((1,), -10.0, 10.0))
b  = tf.Variable(tf.random.uniform((1,), -10.0, 10.0))

In [9]:
learning_rate = tf.Variable(0.001) # learning rate는 0.001로 할까

In [15]:
for i in range(1000+1): # 1001번의 for문을 돌린다.
    with tf.GradientTape() as tape: # with 문 안의 함수를 tape에 기록함
        hypothesis = W1 * x1 + W2 * x2 + b
        cost = tf.reduce_mean(tf.square(hypothesis - y))
    W1_grad, W2_grad, b_grad = tape.gradient(cost, [W1, W2, b]) # cost함수 내의 W1, W2, b 의 미분값을 왼쪽으로 삽입
    W1.assign_sub(learning_rate * W1_grad) # W1에 learning_rate * 지금 W1의 기울기를 넣기
    W2.assign_sub(learning_rate * W2_grad) # W2에 learning_rate * 지금 W2의 기울기를 넣기
    b.assign_sub(learning_rate * b_grad)   # b에  learning_eate * b의 기울기를 넣기
    
    # 한 번 결과를 볼까
    
    if i % 50 == 0: # 50번에 한 번씩 아래로 ㄱㄱ
        print("{:5} | {:10.6f} | {:10.4f} | {:10.4f} | {:10.6f}".format(
          i, cost.numpy(), W1.numpy()[0], W2.numpy()[0], b.numpy()[0]))

    0 | 1256.619385 |    -9.1905 |    -1.9179 |  -8.748963
   50 | 287.577789 |    -3.0720 |    -0.2124 |  -6.530516
  100 |  69.545204 |    -0.2508 |     0.7721 |  -5.404896
  150 |  19.368086 |     1.0387 |     1.3499 |  -4.809908
  200 |   7.329938 |     1.6190 |     1.6926 |  -4.475818
  250 |   4.214845 |     1.8726 |     1.8963 |  -4.272120
  300 |   3.292675 |     1.9766 |     2.0163 |  -4.134976
  350 |   2.951777 |     2.0129 |     2.0849 |  -4.032729
  400 |   2.781655 |     2.0190 |     2.1218 |  -3.949406
  450 |   2.668293 |     2.0120 |     2.1388 |  -3.876784
  500 |   2.576570 |     1.9996 |     2.1433 |  -3.810526
  550 |   2.494502 |     1.9851 |     2.1400 |  -3.748281
  600 |   2.417632 |     1.9701 |     2.1318 |  -3.688743
  650 |   2.344183 |     1.9551 |     2.1204 |  -3.631163
  700 |   2.273401 |     1.9403 |     2.1072 |  -3.575097
  750 |   2.204935 |     1.9257 |     2.0929 |  -3.520278
  800 |   2.138608 |     1.9115 |     2.0779 |  -3.466537
  850 |   2.0

### 행렬식(Matrix)

In [16]:
tf.random.set_seed(0)

In [18]:
x = [
    [1., 0., 3., 0., 5.],  # 행렬 형태로 데이터를 저장
    [0., 2., 0., 4., 0.]
]
y = [1, 2, 3, 4, 5]

In [19]:
W = tf.Variable(tf.random.uniform((1, 2), -1.0, 1.0)) # (1, 2) # 1행 2열
b  = tf.Variable(tf.random.uniform((1, ), -1.0, 1.0))

In [20]:
learning_rate = tf.Variable(0.001)

In [22]:
for i in range(1000+1):
    with tf.GradientTape() as tape:
        hypothesis = tf.matmul(W, x) + b # 3장에서도 언급했듯, matmul은 행렬곱이다.
        cost = tf.reduce_mean(tf.square(hypothesis - y))
        
        W_grad, b_grad = tape.gradient(cost, [W, b])
        W.assign_sub(learning_rate * W_grad)
        b.assign_sub(learning_rate * b_grad)
        
    if i % 50 == 0:
        print("{:5} | {:10.6f} | {:10.4f} | {:10.4f} | {:10.6f}".format(
          i, cost.numpy(), W.numpy()[0][0], W.numpy()[0][1],b.numpy()[0]))        

    0 |   0.067913 |     0.8377 |     0.8072 |   0.617546
   50 |   0.065871 |     0.8401 |     0.8102 |   0.608213
  100 |   0.063892 |     0.8425 |     0.8131 |   0.599015
  150 |   0.061971 |     0.8449 |     0.8159 |   0.589952
  200 |   0.060109 |     0.8473 |     0.8187 |   0.581024
  250 |   0.058302 |     0.8496 |     0.8215 |   0.572229
  300 |   0.056550 |     0.8519 |     0.8242 |   0.563567
  350 |   0.054850 |     0.8541 |     0.8269 |   0.555034
  400 |   0.053202 |     0.8563 |     0.8295 |   0.546631
  450 |   0.051603 |     0.8585 |     0.8321 |   0.538354
  500 |   0.050052 |     0.8606 |     0.8346 |   0.530203
  550 |   0.048548 |     0.8627 |     0.8371 |   0.522174
  600 |   0.047088 |     0.8648 |     0.8396 |   0.514268
  650 |   0.045673 |     0.8669 |     0.8420 |   0.506481
  700 |   0.044300 |     0.8689 |     0.8444 |   0.498812
  750 |   0.042969 |     0.8709 |     0.8468 |   0.491259
  800 |   0.041678 |     0.8728 |     0.8491 |   0.483820
  850 |   0.04